**Ex. 2 DMRG implementation**

We start by creating a random MPS and right-normalizing it. We also need the MPO of the transverse field Ising Hamiltonian. These are two ingredients needed as an input to the actual DMRG.

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import math
from scipy import linalg
import scipy.sparse
from scipy.sparse import linalg

from numpy import transpose as tr, conjugate as co
from scipy.linalg import expm, svd
from scipy.sparse.linalg import eigsh, LinearOperator
import math


Let us start with some utility functions to left and right nomalize the MPS at a site. This is the same as last week's exercise (Ex. 2) -- the code is given to you.

In [22]:
def dot(A,B):
    """ Does the dot product like np.dot, but preserves the shapes also for singleton dimensions """
    s1 = A.shape
    s2 = B.shape
    return np.dot(A,B).reshape((s1[0],s2[1]))

def right_canonize(M1,M2,return_S = False):
    """ Right normalizes M2 into B matrix, M1 loses its canonization """
    s, da, db = M2.shape
    U, S, Vh = svd(M2.transpose((1,0,2)).reshape((da,s*db)))
    #this reshapes M2 and finds its svd
    B2     = Vh.reshape((Vh.shape[0],s,db)).transpose((1,0,2))[:,:da,:]
    M1     = np.tensordot(M1,dot(U[:,:min(da,np.shape(S)[0])],np.diag(S[:min(da,np.shape(S)[0])])),axes=((2),(0)))
    if return_S:
        return M1, B2, S
    else:
        return M1, B2

    
def left_canonize(M1,M2,return_S = False):
    """ Left normalizes M1 into A matrix, M2 loses its canonization"""
    s, da, db = M1.shape
    U, S, Vh = svd(M1.reshape((s*da,db)))
    A1      = U.reshape((s,da,U.shape[1]))[:,:,:min(db,np.shape(S)[0])]
    M2     = np.tensordot(dot(np.diag(S[:min(db,np.shape(S)[0])]),Vh[:min(db,np.shape(S)[0]),:]),M2,axes=((1),(1)))
    if return_S:
        return A1, M2, S
    else:
        return A1, M2


The MPO for the TFIM Hamiltonian is the same as in Ex. 2 of this week.

In [23]:
sigma_x=np.array([[0,1],[1,0]])
sigma_y=np.array([[0+0j,0-1j],[0+0j,0+1j]])
sigma_z=np.array([[1,0],[0,-1]])

In [24]:
def make_Ising(N,h):
    """
    N: number of spins, h: transverse field field
    return a list of MPO [O1, ...ON] with dim(Oi)=(s,s,da,db); here s=2 and da=db=3
    """

    O1 = np.zeros((2,2,1,3))
    O1[:,:,0,0]=-h*sigma_x
    O1[:,:,0,1]=sigma_z
    O1[:,:,0,2]=np.eye(2)

    O2=np.zeros((2,2,3,3))
    O2[:,:,0,0]=np.eye(2)
    O2[:,:,1,0]=sigma_z
    O2[:,:,2,0]=-h*sigma_x
    O2[:,:,2,1]=sigma_z
    O2[:,:,2,2]=np.eye(2)

    O3=np.zeros((2,2,3,1))
    O3[:,:,2,0]=-h*sigma_x
    O3[:,:,1,0]=sigma_z
    O3[:,:,0,0]=np.eye(2)
    O_Ising=[O1]
    for i in range(N-2):
        O_Ising.append(O2)
    O_Ising.append(O3)
    return O_Ising


We will now need functions that form right and left environments on the chain. Starting from the right of the chain, we contract indices up to some vertical cut and obtain $R^i_{a_i a'_i b_i}$ environment tensor that has two MPS virtual indices $a_i$ and $a'_i$ as well as one MPO index $b_i$. Note that we should work here with the right-normalized MPS representation. The recurrent formula for $R^{i + 1}_{a_{i + 1} a'_{i + 1} b_{i + 1}}$ is straightforward:
$$R^{i + 1}_{a_{i + 1} a'_{i + 1} b_{i + 1}} = \sum\limits_{\sigma_i \sigma'_i} \sum\limits_{b_i a_i a'_i} R^i_{a_i a'_i b_i} B^{a_{i + 1}}_{\sigma_i a_i} B^{\dagger a'_{i + 1}}_{\sigma'_i a'_i} W^{\sigma_i \sigma'_i}_{b_i b_{i + 1}}.$$

The same procedure applies to the construction of left environments $L^i_{a_i a'_i b_i}$, which are constructed from MPO $W_i$ and left-normalized MPS representation $A$.

In [29]:
# ---- R^{i + 1}     -- B^{\dag} -- R^i
#      R^{i + 1}          |         R^i
# ---- R^{i + 1} === ---- W ------- R^i
#      R^{i + 1}          |         R^i
# ---- R^{i + 1}     ---- B ------- R^i

def add_site_to_R_env(R_env, B, W):
    """
    R_env: right environment from previous step; shape (da_psi, da_H, da_psi) 
    B: right-normalized; shape (s, db_psi, da_psi)
    W: MPO; shape (s, s, db_H, da_H)
    
    Returns
    R_env: updated right environment; shape (db_psi, db_H, db_psi)
    """
    
    # implement contractions (either np.tensordot or np.einsum)
    R_env2 = R_env
    
    R_env = np.tensordot(B.conj(), R_env, axes=(2, 0)) #new: s, db_psi, da_H, da_psi
    R_env = np.tensordot(W, R_env, axes=([0,3],[0,2])) #s and da_H vs s and da_H -> new s,db_H,db_psi,da_psi
    R_env = np.tensordot(B, R_env, axes=([0,2],[0,3])) #s and db_psi vs s and da_psi -> new db_psi, db_H, db_psi
    R_env = np.transpose(R_env, (2, 1, 0))

    
    # Method 2: using np.einsum
    # contract a_i' (index r in einsum)
    R_env2 = np.einsum('slr,rab->slab', B.conj(), R_env2)
    # contract b_i (index d in einsum) and sigma'_i (index a in einsum)
    R_env2 = np.einsum('abcd,axdy->bcxy', W, R_env2)
    # contract sigma_i and a_i
    R_env2 = np.einsum('abc,axyc->bxy', B, R_env2)
    # transpose to get the correct shape
    R_env2 = np.transpose(R_env2, (2, 1, 0))

    # R_env and R_env2 should be the same
    if  np.allclose(R_env, R_env2, atol=1e-10) is False:
        print("Inconsistency! ", R_env, R_env2)

    

    return R_env    

#  L^i -- A^{\dag} ---     L^{i + 1} ---
#  L^i       |             L^{i + 1}
#  L^i ------W ------- === L^{i + 1} ---
#  L^i       |             L^{i + 1}
#  L^i ------A -------     L^{i + 1} ---

def add_site_to_L_env(L_env, A, W):
    """
    L_env: left environment from previous step; shape (da_psi, da_H, da_psi) 
    A: left-normalized; shape (s, da_psi, db_psi)
    W: MPO; shape (s, s, da_H, db_H)
    
    Returns
    L_env: updated left environment; shape (db_psi, db_H, db_psi)
    """
        
    # implement contractions (either np.tensordot or np.einsum)
    L_env2 = L_env

    # Methods 1 and 2 are equivalent
    # Method 1: using np.tensordot                              ##########Why when conj we dont swapo the indices?##########
    L_env2 = np.tensordot(A.conj(), L_env2, axes=(1, 0)) # s, da_psi, db_psi and da_psi, da_H, da_psi -> new: s, dp_psi, da_H, da_psi
    L_env2 = np.tensordot(W, L_env2, axes=([0,2],[0,2])) # s, s, da_H, db_H and s, dp_psi, da_H, da_psi -> new: s, db_H, db_psi, da_psi
    L_env2 = np.tensordot(A, L_env2, axes=([0,1],[0,3])) # s, da_psi, db_psi and s, db_H, db_psi, da_psi -> new: db_psi, db_H, db_psi           ##########Why contract with da and not db? why not [0,2][0,2]##########
    L_env2 = np.transpose(L_env2, (2, 1, 0)) ##########Why transpose? Is it not equal?##########

    # Method 2: using np.einsum
    L_env = np.einsum('slr,lxy->srxy', A.conj(), L_env)
    L_env = np.einsum('axby,awbz->xywz', W, L_env)
    L_env = np.einsum('slr,sxyl->rxy', A, L_env)
    L_env = np.transpose(L_env, (2, 1, 0))

    # L_env and L_env2 should be the same
    if  np.allclose(L_env, L_env2, atol=1e-10) is False:
        print("Inconsistency! ", L_env, L_env2)

    return L_env2

In the DMRG algorithm, one locally solves the eigenvalue problem $$\sum\limits_{\sigma_l a_{l - 1} a_l} \left(\underbrace{\sum\limits_{b_l b_{l - 1}} L_{a_{l - 1} b_{l - 1} a'_{l - 1}} W^{\sigma'_l \sigma_l}_{b_{l - 1} b_l} R_{a_l b_l a'_l}}_{\hat H}\right) M^{\sigma_l}_{a_{l - 1} a_l} = \lambda M^{\sigma'_l}_{a'_{l - 1} a'_l}.$$

The underbraced expression is the local "Hamiltonian" acting on the MPS matrix at site $l$. The construction of this Hamiltonian itself is very inefficient, so one only defines the *action* of this Hamiltonian as a Linear Operator. (Recall this sort of procedure from Exercise Sheet 3, Question 1B.)

In [30]:
def H_local(L_env, W, R_env, M):
    """
    L_env: left environment up to site l-l
    W: MPO at site l
    R_env: right environment from site l+1
    M: MPS matrix at site l
    """
    
    # distinguish between the boundary and the bulk
    s = 2    # dimension of the physical leg
    if len(M.shape) == 1:  # in case of the boundary
        flatten = True
        M = np.reshape(M, (s, L_env.shape[0], R_env.shape[0]))
    elif len(M.shape) == 3:  # in case of the bulk index
        flatten = False
    else:
        raise ValueError('Unknown format for M')
    
    # L -- (blank) --- R
    # L    |           R
    # L -- W --------- R
    # L    |           R
    # L -- M --------- R
    
    # contract L_env, M, W and R_env as shown above                                 ##########Why this order (W,hpsi) and tipps for inidce overview?##########
    hpsi = np.einsum('abc,scd->absd', L_env, M)     # contract a_{l - 1}
    hpsi = np.einsum('abcd,xcby->adxy', W, hpsi)    # contract b_{l} and \sigma_l
    hpsi = np.einsum('abcd,xbd->acx', hpsi, R_env)  # contract b_{l - 1} and a_{l}
    if flatten:
        return hpsi.flatten()
    return hpsi

Now we are ready to run DMRG. For a given value of $h$ transverse field, we construct the MPO, create a random initial MPS, right-normalize it, precompute right environments.

Then the algorithm performs consequtive right and left "sweeps". Starting from the left, we compute local left environment, construct local "Hamiltonian", optimize MPS locally and normalize.


In [33]:
def run_DMRG(h, L, chi = 60, nmax = 1000, verbose = False, atol=1e-12):
    """
    h: transverse field
    L: number of spins
    chi: maximum bond dimension
    nmax: maximum number of left and right sweeps

    returns:
    energy: ground state energy
    Lambdas: list of singular value matrices
    """
    # construct the MPO
    W = make_Ising(L,h)

    s = 2

    # define random MPS
    M = [np.random.standard_normal((s, min(chi, s ** i, s ** (L - i)),
                                       min(chi, s ** (i + 1), s ** (L - 1 - i)))) for i in range(L)]

    # right-normalize the MPS
    for i in range(L - 1, 0, -1):
        M[i - 1], M[i] = right_canonize(M[i - 1], M[i])

    # to run on the left boundary, introduce fake matrix that we discard later
    _, M[0] = right_canonize(np.ones((1, 1, 1)), M[0])

    ## compute right environments
    R_environments = [np.ones((1, 1, 1))]    # again start with a fake matrix on the right boundary
    for i in range(1, L):
        R_environments.append(add_site_to_R_env(R_environments[i - 1], M[-i], W[-i]))

    local_energy_previous = np.inf
    delta = np.inf

    for n in range(nmax):
        ## right sweep
        L_environments = [np.ones((1, 1, 1))]  # again start with a fake matrix on the left boundary

        #Lambdas = []
        for i in range(L - 1):
            # compute the local H dimension
            local_H_dim = s * L_environments[i].shape[0] * R_environments[L - 1 - i].shape[0]

            # construct the LinearOperator
            Hop = scipy.sparse.linalg.LinearOperator((local_H_dim, local_H_dim), \
                matvec = lambda M: H_local(L_environments[i], W[i], R_environments[L - 1 - i], M))

            # obtain best local MPS (linearized) and local energy using the Lanczos algorithm
            energy, V = scipy.sparse.linalg.eigsh(Hop, k = 1, v0 = M[i].flatten(), \
                                                  tol = 1e-2 if n < 2 else 0, which = 'SA')
            energy = energy[0]
            if energy > local_energy_previous:
                success = False
            delta = energy - local_energy_previous
            local_energy_previous = energy

            # reshape the obtained result to the shape of the MPS matrix
            M[i] = V.reshape((s, L_environments[i].shape[0], R_environments[L - 1 - i].shape[0]))

            # left-normalize the result (it will affect M[i + 1], but we will optimize it the next step)
            M[i], M[i + 1], Lambda = left_canonize(M[i], M[i + 1], return_S = True)

            L_environments.append(add_site_to_L_env(L_environments[i], M[i], W[i]))

        ## repeat the same for left sweep
        R_environments = [np.ones((1, 1, 1))]
        Lambdas = []
        for i in range(L - 1, 0, -1):
            local_H_dim = s * L_environments[i].shape[0] * R_environments[L - 1 - i].shape[0]

            Hop = scipy.sparse.linalg.LinearOperator((local_H_dim, local_H_dim), \
                                 matvec = lambda M: H_local(L_environments[i], W[i], \
                                                            R_environments[L - 1 - i], M))

            energy, V = scipy.sparse.linalg.eigsh(Hop, k = 1, v0 = M[i].flatten(), \
                                                  tol = 1e-2 if n < 2 else 0, which = 'SA')
            energy = energy[0]
            if energy > local_energy_previous:
                success = False
            delta = energy - local_energy_previous
            local_energy_previous = energy
            M[i] = V.reshape((s, L_environments[i].shape[0], R_environments[L - 1 - i].shape[0]))
            M[i - 1], M[i], Lambda = right_canonize(M[i - 1], M[i], return_S = True)
            Lambdas.append(Lambda)
            R_environments.append(add_site_to_R_env(R_environments[L - 1 - i], M[i], W[i]))

        # === check convergence ===
        if verbose:
            print(h, n, 'dE = ', abs(delta))
        if abs(delta) < atol:
            if verbose:
                print("Converged after {:d} sweeps!".format(n))
                print('Ground-state energy: {:.10f}'.format(energy))
            break

    if abs(delta) > atol:
        print("Convergence not reached after  {:d} sweeps!".format(n))
        print(h, n, 'dE = ', abs(delta))

    return energy, Lambdas

 Now, as you obtained the working version of DMRG, you should compare it with the exact diagonalization (ED) data. Consider the $L = 8$ chain with open boundary conditions and validate that the ground state energy at $h/J = 2$ equals -16.88514149.

In [34]:
run_DMRG(2, 8, 16)[0]

np.float64(-16.885141493208167)